In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

from hydra import mean_embeddings,get_model,well_plotly
import pickle

In [ ]:
# Función con la que se crean las graficas 3D mostradas en el documento 
well_plotly(datos_margento.replace({'organic matter':"Materia orgánica",
                            'loamy sand':"Arena arcillosa",
                            'sandy clay':"Arcilla arenosa",
                            'conglomerate':"Conglomerado",
                            'sand':"Arena",
                            'clay':"Arcilla",
                            'silt':"Limo"}),
    x="x",y="y",z="z",clase="clase",z_scale=-1,titulo="Perforaciones de la zona de estudio",save_html=False,text="nombre_pozo")

In [ ]:
#Embedding preentrenado con textos de geociencias  
with open("archivos_base\Glove_300.pkl","rb") as data:
    modelEmb=pickle.load(data)

In [ ]:
# Transforma las descripciones en vectores y obtiene el vector promedio de la descripción 
datos_margento=mean_embeddings(datos_margento,modelEmb)

In [ ]:
# Red utilizada para transformar vectores en clases/etiquetas   
model = Sequential()
model.add(Dense(100, input_dim=300, activation='relu'))
model.add(Dense(100, activation='relu'))
model.add(Dense(100, activation='relu'))
model.add(Dense(units=len(datos_margento.clase.unique()), activation='softmax'))
model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])
model.summary()

In [ ]:
# Clase creada para facilitar el proceso de interpolación 3D, en este se especifica la distancia espacial entre 
# puntos y el número de franjas verticales
modelo_3D=get_model(datos_margento[datos_margento["z"]<16],
                    'DEM_Recorte1.tif',
                    intervalo_xy=50,
                    franjas_z=16,
                    x="x",
                    y="y",
                    z="z",
                    test_size=0.1)


In [ ]:
# Función para interpolación lineal, es aplicable tanto para los datos de OneHotEncode 
# como para el ventor promedio resultante del embedding 
modelo_3D.interpolacion_lineal_basica(objetivo="mean",obj_type="vector")

In [ ]:
# se traducen los vectores interpolados con la red neuronal entrenada para obtener 
# la clase a la cual pertenece cada punto en el espacio   
res_inter=modelo_3D.res_lineal_basica.copy()
res_inter["clase_decode"]=encode.inverse_transform(model.predict(np.vstack(res_inter['clase'].values)))
